In [1]:

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
from keras.optimizers import Adagrad
import random
from keras.optimizers import Adam

In [2]:
# tokenizing and lematizing
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
# lemmatize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique lemmatized words", words)
pickle.dump(words,open('texts.pkl','wb'))
pickle.dump(classes,open('labels.pkl','wb'))
training = []
output_empty = [0] * len(classes)

283 documents
129 classes ['What are the types of depression?', 'about', 'afternoon', 'anxious', 'ask', 'at what age does anxiety peak?', 'can lack of sleep make you feel sad?', 'can low blood sugar cause suicidal thoughts?', 'casual', 'creation', 'death', 'default', 'depressed', 'do we control our thoughts?', 'does oversleeping cause depression?', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'how can we reduce anxiety?', 'how does depression affect the world?', 'how long can anxiety last?', 'how many thoughts a day do we have?', 'i am a victim of bullying', 'i am afraid i will fail again', 'i am afraid to file 

In [3]:

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])

In [4]:

random.shuffle(training)
training = np.array(training)
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


Training data created


C:\Users\Asus\AppData\Local\Temp\ipykernel_22788\2119147192.py:3: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [5]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


In [6]:
#sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
#model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])
#only get 77acc


In [7]:
#NADAM OPTIMIZER TO GET 87 acc

#from keras.optimizers import Nadam
#optimizer = Nadam(learning_rate=0.002)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [8]:
#adam optimizer to get 86 acc

#from keras.optimizers import Adam
#optimizer = Adam(learning_rate=0.001)
#model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [14]:
# ADAGRAD OPTIMIZER TO GET 88 acc


#from keras.optimizers import Adadelta
#optimizer = Adadelta(learning_rate=1.0)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [16]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
57/57 [==============================] - 0s 4ms/step - loss: 0.4809 - accuracy: 0.8481
Epoch 2/200
57/57 [==============================] - 0s 4ms/step - loss: 0.5384 - accuracy: 0.8233
Epoch 3/200
57/57 [==============================] - 0s 3ms/step - loss: 0.5853 - accuracy: 0.8057
Epoch 4/200
57/57 [==============================] - 0s 3ms/step - loss: 0.5258 - accuracy: 0.8551
Epoch 5/200
57/57 [==============================] - 0s 3ms/step - loss: 0.4516 - accuracy: 0.8657
Epoch 6/200
57/57 [==============================] - 0s 3ms/step - loss: 0.5627 - accuracy: 0.8198
Epoch 7/200
57/57 [==============================] - 0s 4ms/step - loss: 0.4722 - accuracy: 0.8375
Epoch 8/200
57/57 [==============================] - 0s 3ms/step - loss: 0.4428 - accuracy: 0.8763
Epoch 9/200
57/57 [==============================] - 0s 3ms/step - loss: 0.4843 - accuracy: 0.8481
Epoch 10/200
57/57 [==============================] - 0s 3ms/step - loss: 0.5255 - accuracy: 0.8269
Epoch 11/

In [17]:
#model.save('nadam_model_87.h5', hist)
#print("model created")